In [151]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
from selenium.webdriver.common.keys import Keys
import time
from selenium.common.exceptions import NoSuchAttributeException
import json


In [270]:
def gopher_broke(driver:webdriver, url:str, query:str, location:str):
    try:
        driver.get(url)
        
        time.sleep(2)
        
        keyword_input = driver.find_element(By.ID, "typeaheadInput")
        keyword_input.clear()
        keyword_input.send_keys(query)
        
        location_input = driver.find_element(By.ID, "google-location-search")
        location_input.clear()
        location_input.send_keys(location)
        
        location_input.send_keys(Keys.RETURN)
        
        time.sleep(5)
        
        
        data = BeautifulSoup(driver.page_source, 'html.parser')
        driver.quit()
        
        desc = data.find('div', class_="m-px mx-auto max-w-screen-2xl sm:px-6")
        jobs = desc.div.find_all('div')
        info = desc.div.find_all('div', class_='content')
        head = desc.div.find_all('div', class_='header')
        links:list = []
        descriptions:list = []
        companies:list = []
        job_types:list = []
        salaries:list = []
        easy_apply:list = []
        for listing in jobs:
            if listing.find('div') is None:
                continue
            job_card = listing.find('div')
            if job_card.a is None:
                continue
            links.append(listing.div.a['href'])
        for item in info:
            descriptions.append(item.find('p', class_='line-clamp-2 h-10 shrink grow basis-0 text-sm font-normal text-zinc-900').text)
            job_type_and_salary = item.find_all('p', class_='m-0 flex h-full w-full items-center justify-center overflow-hidden text-ellipsis text-xs font-medium text-zinc-600')
            
            if len(job_type_and_salary) == 2:
                easy_apply.append(job_type_and_salary[0].text.lower() == 'easy apply')
                job_types.append(job_type_and_salary[0])
                salaries.append(job_type_and_salary[1])
                continue
            if len(job_type_and_salary) == 3:
                easy_apply.append(job_type_and_salary[0].text.lower() == 'easy apply')
                job_types.append(job_type_and_salary[1])
                salaries.append(job_type_and_salary[2])
                continue
            easy_apply.append(job_type_and_salary[0].text.lower() == 'easy apply')
            job_types.append(job_type_and_salary[0])
            salaries.append('Not Provided')
        for company in head:
            companies.append(company.span.text)
        return {'company': companies, 'links' : links, 'description' : descriptions, 'job_type' : job_types, 'salary' : salaries, 'easy_apply' : easy_apply}
    except NoSuchAttributeException as nsae:
            print(nsae)

In [271]:
new_driver = webdriver.Chrome()
data = gopher_broke(driver=new_driver, url='https://www.dice.com', query='data scientist',location='orlando, fl')

In [272]:
data_chunks = [set(data['links'][idx:idx + 4]) for idx in range(0,len(data['links']), 4)]

In [273]:
data['links'] = data_chunks

In [274]:
df = pd.DataFrame(data)
df

,company,links,description,job_type,salary,easy_apply
0,Kforce Technology Staffing,{/company-profile/171741b7-87ab-5a96-90bf-be5a...,RESPONSIBILITIES: Kforce has a client in Altam...,"[Contract, Third Party]",[$50 - $55 hourly],False
1,FlairTech Solutions,{https://www.dice.com/job-detail/1a06d3c5-a451...,Role: Data Scientist Location: Remote Duration...,"[Contract, Third Party]",[Depends on Experience],True
2,SSI People,{https://www.dice.com/job-detail/f753e8dc-3dba...,Job Description: About the Role We are seeking...,[Contract],[$40 - $46],True
3,"KPI Partners, Inc.",{https://www.dice.com/job-detail/0608171c-3eef...,"KPI Partners, A global consulting firm focused...",[Contract],[$90 - $100],True
4,Lockheed Martin Corporation,{/company-profile/f2ad2a59-bdb4-487c-bd0d-19f0...,Job Description You will be the Data scientis...,[Full-time],"[USD 104,500.00 - 184,115.00 per year]",False
5,PeakIT,{https://www.dice.com/job-detail/0afbe516-da15...,Computer vision model experiences is a must. W...,[Contract],[$90 - $90],True
6,BP,{/company-profile/a6e96ccc-791a-5c39-990e-47d2...,Entity: Production & Operations Job Family G...,[Full-time],"[USD 143,000.00 - 180,000.00 per year]",False
7,Kforce Technology Staffing,{https://www.dice.com/job-detail/b08a798b-e8a0...,RESPONSIBILITIES: Kforce has a client that is ...,[Contract],[$40 - $50 hourly],False
8,Aditi Consulting,{/company-profile/b23f065d-8d4d-53a1-b4e9-08a0...,Payrate: $70.50 - $80.59/hr. Summary: The Comp...,[Contract],[$70.5 - $80.59 hr],True
9,Karwell Technologies Inc,{https://www.dice.com/job-detail/b7b3043d-f2e9...,#W2 Requirements Job Title: Data Scientist L...,[Contract],[$65 - $69],True
